In [52]:
import os
import pandas as pd
import csv
from ete3 import NCBITaxa
import biothings_client as bt
import tarfile
import gzip

In [3]:
os.getcwd()

'/Users/bailinzhang/Documents/Wu_Lab/Projects/GMMAD2'

In [4]:
micro_disease_df = pd.read_csv(os.path.join("downloads", "disease_species.csv"), low_memory=False)
micro_meta_df = pd.read_csv(os.path.join("downloads", "micro_metabolic.csv"), low_memory=False)
meta_gene_df = pd.read_csv(os.path.join("downloads", "meta_gene_net.csv"), low_memory=False)

ParserError: Error tokenizing data. C error: Expected 24 fields in line 4, saw 29


In [5]:
with open(os.path.join("downloads", "disease_species.csv"), "r") as f:
    for i, line in enumerate(f, start=1):
        if i == 4:
            print(len(line.strip().split(",")))
            print(line)
            break

29
3,D009765,Obesity,Azorhizobium caulinodans,species,7,0,0,0,0,D006262,Health,3,0.029020807,0.0248571,0.031027083,-0.0248571,Decrease,A status with BODY WEIGHT that is grossly above the acceptable or desirable weight, usually due to accumulation of excess FATS in the body. The standards may vary with age, sex, genetic or cultural background. In the BODY MASS INDEX, a BMI greater than 30.0 kg/m2 is considered obese, and a BMI greater than 40.0 kg/m2 is considered morbidly obese (MORBID OBESITY).,Pseudomonadota,Alphaproteobacteria,Hyphomicrobiales,Xanthobacteraceae,Azorhizobium



In [8]:
def line_generator(in_file):
    """generates lines from a CSV file, yielding each line as a list of strings
    This function opens the specified CSV file, skips the header row, and yields each subsequent line as a list of strings.

    :param in_file: The path to the CSV file.
    :return: An iterator that yields each line of the CSV file as a list of strings.
    """
    with open(in_file, "r") as in_f:
        reader = csv.reader(in_f)
        next(reader)
        for line in reader:
            yield line

In [10]:
micro_disease_path = os.path.join("downloads", "disease_species.csv")
micro_disease = line_generator(micro_disease_path)
for i, line in enumerate(micro_disease):
    if i == 3:
        print(len(line))
        print(line)
    elif i == 15:
        print(len(line))
        print(line)
    elif i == 16:
        print(len(line))
        print(line)
        break

26
['4', 'D008103', 'Liver Cirrhosis', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease', 'Liver disease in which the normal microcirculation', ' the gross vascular anatomy', ' and the hepatic architecture have been variably destroyed and altered with fibrous septa surrounding regenerated or regenerating parenchymal nodules.', 'Pseudomonadota', 'Alphaproteobacteria', 'Hyphomicrobiales', 'Xanthobacteraceae', 'Azorhizobium']
30
['16', 'D001172', 'Arthritis', ' Rheumatoid', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease', 'A chronic systemic disease', ' primarily of the joints', ' marked by inflammatory changes in the synovial membranes and articular structures', ' widespread fibrinoid degeneration of the collagen fibers in mesenchymal tissues', ' and by atrophy and rarefact

In [24]:
def line_generator(in_file):
    """Yield each CSV line as a list of exactly 24 fields,
    rejoining commas inside the 'disease' and 'disease_info' columns.
    """
    EXPECTED_COUNT = 24
    HEAD_COUNT = 2
    TAIL_COUNT = 5
    FIXED_COUNT = 15

    with open(in_file, "r", encoding="utf-8") as f:
        header = next(f).rstrip("\n").split(",")
        print(f"Header column names:{header}")
        next(f)
        for line in f:
            parts = [part.strip() for part in line.split(",")]
            if len(parts) == EXPECTED_COUNT:
                yield parts
            else:
                alteration_idx = next(i for i, p in enumerate(parts) if p in ("Increase", "Decrease"))
                head = parts[:HEAD_COUNT]
                fixed_start = alteration_idx - (FIXED_COUNT - 1)
                disease = ",".join(parts[2:fixed_start])
                fixed = parts[fixed_start: alteration_idx + 1]
                disease_info = ",".join(parts[alteration_idx + 1: len(parts) - TAIL_COUNT])
                tail = parts[-TAIL_COUNT:]
                new_line = head + [disease] + fixed + [disease_info] + tail
                assert len(new_line) == EXPECTED_COUNT, f"Expected {EXPECTED_COUNT} cols, got {len(new_line)}"
                yield new_line


In [25]:
micro_disease = line_generator(micro_disease_path)
for i, line in enumerate(micro_disease):
    if i == 3:
        print(len(line))
        print(line)
    elif i == 15:
        print(len(line))
        print(line)
    elif i == 16:
        print(len(line))
        print(line)
        break

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']
24
['5', 'D001327', 'Autoimmune Diseases', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease', 'Disorders that are characterized by the production of antibodies that react with host tissues or immune effector cells that are autoreactive to endogenous peptides.', 'Pseudomonadota', 'Alphaproteobacteria', 'Hyphomicrobiales', 'Xanthobacteraceae', 'Azorhizobium']
24
['17', 'D043183', 'Irritable Bowel Syndrome', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease

In [28]:
taxids = [line[5] for line in line_generator(micro_disease_path)]
print(len(set(taxids)))

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']
6966


In [35]:
def get_taxon_info(f_path) -> list:
    """retrieves taxonomic information for a given list of taxon IDs from disease_species.csv

    This function reads taxon IDs, removes duplicates, and queries taxonomic info from biothings_client
    to retrieve detailed taxonomic information including scientific name, parent taxid, lineage, and rank.

    :param f_path: Path to disease_species.csv containing the taxids.
    :return: A list of dictionaries containing taxonomic information.
    """
    taxids = [line[5] for line in line_generator(f_path)]
    taxids = set(taxids)
    t = bt.get_client("taxon")
    taxon_info = t.gettaxa(taxids, fields=["scientific_name", "parent_taxid", "lineage", "rank"])
    return taxon_info

In [36]:
notfound = [
    taxon["query"]
    for taxon in get_taxon_info(micro_disease_path)
    if "notfound" in taxon.keys()
]

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']


In [37]:
len(notfound)

95

In [40]:
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()

Local taxdump.tar.gz seems up-to-date


Loading node names...
2684668 names loaded.
418698 synonyms loaded.
Loading nodes...
2684668 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /Users/bailinzhang/.etetoolkit/taxa.sqlite ...
 2684000 generating entries... 
Uploading to /Users/bailinzhang/.etetoolkit/taxa.sqlite


Inserting synonyms:      60000 

Inserting taxids:       40000   

Inserting taxids:       2680000    

In [118]:
def load_merged_from_tar(tar_gz_path, f_name="merged.dmp"):
    """Parse 'merged.dmp' of taxdump.tar.gz downloaded by ete3.
    Returns a dict {old_taxid: new_taxid}.
    """
    if not os.path.exists(tar_gz_path):
        ncbi = NCBITaxa()
        ncbi.update_taxonomy_database()

    taxid_mapping = {}
    with tarfile.open(tar_gz_path, 'r:gz') as tar:
        f = tar.getmember(f_name)
        with tar.extractfile(f) as fp:
            for line in fp:
                parts = line.decode('utf-8').split('\t')
                old, new = parts[0], parts[2]
                taxid_mapping[old] = new
    return taxid_mapping

In [119]:
def get_current_taxid(old_taxids, merged_mapping):
    taxid_mapping = {}
    for old_taxid in old_taxids:
        taxid_mapping[old_taxid] = merged_mapping[old_taxid]
    return taxid_mapping

In [120]:
mapping = load_merged_from_tar("taxdump.tar.gz")
mapped_taxid = get_current_taxid(notfound, mapping)

In [109]:
if "194866" in mapping:
    print("194866 is in the mapping")
    print(f"current taxid for 194866: {mapping['194866']}")
else:
    print("194866 is not in the mapping")

194866 is in the mapping
current taxid for 194866: 46624


In [122]:
len(mapped_taxid)

95